In [51]:
import os

In [72]:
%pwd

'E:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow'

In [53]:
os.chdir("../")

In [59]:
%pwd

'E:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow'

In [56]:
#import os

# Volvamos a mi folder de trabajo para correr este notebook correctamente.
# os.chdir('E:/OneDrive/Documents/End-to-End-MLP-with-MLFlow')


**Creamos un nuevo Tracker URI para este proyecto**
* con esto podremos correr desde este mismo jupyter notebook, la conección con dagshub.

In [60]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Kokit0/End-to-End-MLP-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Kokit0"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1f4de8fcdefd1528df441f1fd62e0391a267d807"

## **Para correr el model de nuevo con otros parámetros (params.yaml), ejecutar desde aquí en adelante** 
**Definamos la entity**


In [61]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig: # Con esto iresmo guardando los parametros, accediendo al path del modelo, la columna objetivo (schem.yaml) y escribiendolo en el URI de MLflow
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


**Preparamos el Configuration Manager**

In [62]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [64]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet #<--- leeremos los parametros para tracker en mi MLflow
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Kokit0/End-to-End-MLP-with-MLFlow.mlflow",
           
        )

        return model_evaluation_config


**Escribamos el ultimo Component**

In [65]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [66]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred): #<--- definimos las Evaluation Metrics aquí. rmse, r2, MAE, etc.
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self): # creamos el df test_data en base al dataset, cargamos el modelo, y generamos el split train/test, usando Dropeo del objetivo

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri) # seteo mi tracker URI para que todo ocurra en mi servidor remoto
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x) #<--- predicted value

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities) #<--- calcula las 3 metricas
            
            # Salvamos metrics en el .json local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae) # así va guardando todos los parámetros en MLflow


            # Con esto verificamos que vaya guardando todo en el URI o en el local drive.
            # (Model registry does not work with file store)
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel") #<--- por ultimo le damos el modelo que queremos (ElasticNet).
            else:
                mlflow.sklearn.log_model(model, "model")

    


### **Creemos el pileline para MLflow**
* y luego corramos todo. 
* ahorap odemos veriricar que se hayan guardado los resultados; metricas y todo en las rutas.

In [67]:
import os
print(os.getcwd())


E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow


In [71]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-29 03:24:42,163: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-29 03:24:42,165: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-29 03:24:42,168: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-29 03:24:42,169: INFO: common: created directory at: artifacts]
[2023-09-29 03:24:42,170: INFO: common: created directory at: artifacts/model_evaluation]
[2023-09-29 03:24:43,133: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


e:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\MLflowenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/09/29 03:24:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.


### **Notas importantes a documentar**: 
* Luego de eliminar artifacts y correr todo desde python main.py, el experimento dejó de generar los resultados en el servidor de mlflow.
* tuve que vovler a correr todo desde el notebook stage_05_model_evaluation.ipynb.
* por motivos desconocidos me generó un par de errores no reconociendo config.yaml o no encontrando el test.csv en artifacts.
* luego de varios intentos volvió a correr y generar version 3 en el servidor mlflow.
* plop!